# Table of number of expected events for various fluxes

This notebook shows how to compute the number of expected events in KM3NeT/ARCA for a given input neutrino flux. In particular, we focus on the fluxes presented in the Supplementary materials of the paper i.e., IceCube Single Power-Law best-fits and various models.

In [1]:
import sys

import numpy as np
import pandas as pd
from scipy.integrate import trapezoid
from scipy.interpolate import interp1d

sys.path.append("../src")
from fluxmethods import SinglePowerLawFlux, SEDFlux

# Inputs

### KM3NeT inputs

The sky-averaged all-flavour $\nu+\bar\nu$ effective area $A_{\rm eff}(E)$ is used to compute the flux needed to get one expected event in KM3NeT, assuming a per-flavour $\nu+\bar\nu$ neutrino spectrum $\Phi(E)$.
$$N_{\rm exp} = 4\pi \times T_{\rm livetime} \times \times 1/2 \times \int A_{\rm eff}(E) \times \Phi(E) {\rm d}E$$

In [2]:
livetime_arca = 335 * 86400  # in seconds
df_aeff_arca = pd.read_json("../data/supplementary/simulations/effective_area_brighttrackselection_allflavour_skyavg.json")
x_aeff_arca, y_aeff_arca = df_aeff_arca["Energy [GeV]"], df_aeff_arca["Aeff [cm^2]"]
f_aeff_arca = interp1d(x_aeff_arca, y_aeff_arca, bounds_error=False, fill_value=0)

energy5_evt, energy50_evt, energy95_evt = 7.24e7, 2.18e8, 2.57e9

In [3]:
models_cosmogenic = [
    "Aloisio+[1505.04020]", "Berat+[2402.04759]", "Boncioli+[1808.07481]", "Condorelli+[2209.08593]",
    "Ehlert+[2304.07321]", "Muzio+[2209.08068]", "Muzio+[2303.04170]", "PAO+[2211.02857]", 
    "Winter+[1901.03338]",  "Zhang+[1812.10289]"
]

models_source = [
    "Boncioli+[1808.07481]_LL-GRB", "Fang+[1311.2044]", "Rodrigues+[2003.08392v3]",
    "Rodrigues+[2307.13024]_Sample_BLLacs", "Rodrigues+[2307.13024]_Sample_SFRQ", "Tamborra+[1504.00107v2]_LL-GRB",
    "Tamborra+[1504.00107v2]_s-GRB", "Winter+[2205.11538v3]_TDE"
]

In [4]:
icecube_singlepowerlaw_bestfit = pd.read_json("../data/external/flux_constraints/icecube_spl_bestfit.json")

# Results

### Table of number of expected events

In [5]:
def nexpected(flux: SEDFlux | SinglePowerLawFlux, energy_range: tuple | None = None) -> float:
    """Compute the number of expected events for a given flux in a given energy range."""
    
    if energy_range is None:
        x = np.logspace(5, 11, 601)
    else:
        x = np.logspace(*np.log10(energy_range), 601)
    y = f_aeff_arca(x) * flux(x)
    
    return trapezoid(y, x=x, axis=0) * (livetime_arca * 4*np.pi)

def get_nexpecteds(flux: SEDFlux | SinglePowerLawFlux) -> float:
    """Return the number of expected events for a given flux, in the full energy range and in the central 90% KM3-230213A neutrinu energy range."""
    
    return (nexpected(flux), nexpected(flux, (energy5_evt, energy95_evt)))

In [6]:
nexps = {}

# Process IceCube single power law best-fit data
for icsample in ["NST", "ESTES", "HESE"]:
    bf = icecube_singlepowerlaw_bestfit[icsample]
    nexps[f"IceCube/{icsample}"] = get_nexpecteds(SinglePowerLawFlux(bf["norm"], bf["gamma"], e0=bf["e0"]))

# Process cosmogenic models
for model in models_cosmogenic:
    nexps[model] = get_nexpecteds(SEDFlux(f"../data/external/flux_models/cosmogenic_{model}.json"))

# Process source models
for model in models_source:
    nexps[model] = get_nexpecteds(SEDFlux(f"../data/external/flux_models/source_{model}.json"))

# Convert results to a Pandas DataFrame
df_nexps = pd.DataFrame.from_dict(nexps, orient='index', columns=["Total Events", "Events in Central 90% Energy Range of KM3-230213A"])

# Display the DataFrame
df_nexps

,Total Events,Events in Central 90% Energy Range of KM3-230213A
IceCube/NST,0.117431,1.425374e-02
IceCube/ESTES,0.052083,3.428880e-03
IceCube/HESE,0.019698,5.090497e-04
Aloisio+[1505.04020],0.469495,1.586657e-02
Berat+[2402.04759],0.001438,3.958025e-04
Boncioli+[1808.07481],0.012525,4.039573e-03
Condorelli+[2209.08593],0.002011,4.576365e-04
Ehlert+[2304.07321],0.242767,2.111172e-01
Muzio+[2209.08068],0.107167,1.011525e-02
Muzio+[2303.04170],0.431498,2.415788e-01
